#Pemrograman Tugas Akhir:


"Segmentasi Burned Area menggunakan Model U-Net pada Citra Landsat 9 (Studi Kasus: Sumatera Selatan)"

*   Data Filtering






# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
# Jalankan ini sekali di Colab untuk install GDAL
!apt install -y gdal-bin python3-gdal


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gdal-bin is already the newest version (3.8.4+dfsg-1~jammy0).
python3-gdal is already the newest version (3.8.4+dfsg-1~jammy0).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


# Overlap 50% Pathrow 124

## Cek Data memiliki citra tidak layak (Piksel bernilai hitam dominan lebih dari 30%)

In [ ]:
import glob
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import os
import pandas as pd

# Path ke folder Multiband
multiband_path = "/content/drive/MyDrive/Data Riset/Data Patch/124/MULTIBAND"
multiband_files = sorted(glob.glob(f"{multiband_path}/*.tif"))

# Inisialisasi hasil
categories = {
    "<30%": 0,
    "30-50%": 0,
    "50-70%": 0,
    ">70%": 0
}

# List data detail
data_list = []

# Proses semua file
for file in multiband_files:
    with rasterio.open(file) as src:
        try:
            img = src.read([5, 4, 3])
            total_pixels = img.shape[1] * img.shape[2]
            black_pixels = np.sum(np.all(img == 0, axis=0))
            black_ratio = black_pixels / total_pixels

            # Tentukan kategori
            if black_ratio < 0.3:
                categories["<30%"] += 1
                cat = "<30%"
            elif black_ratio < 0.5:
                categories["30-50%"] += 1
                cat = "30-50%"
            elif black_ratio < 0.7:
                categories["50-70%"] += 1
                cat = "50-70%"
            else:
                categories[">70%"] += 1
                cat = ">70%"

            # Simpan ke data list
            data_list.append({
                "filename": os.path.basename(file),
                "black_ratio": round(black_ratio * 100, 2),  # persen
                "category": cat
            })
        except:
            print(f" Gagal membaca: {file}")

# Buat DataFrame
df = pd.DataFrame(data_list)

# Tampilkan tabel ringkas
print("\n Statistik Jumlah Citra Berdasarkan Persentase Piksel Dominan Hitam:\n")
stat_table = pd.DataFrame.from_dict(categories, orient='index', columns=['Jumlah Citra'])
stat_table



 Statistik Jumlah Citra Berdasarkan Persentase Piksel Dominan Hitam:



,Jumlah Citra
<30%,504
30-50%,31
50-70%,28
>70%,85


## Menampilkan citra difilter (tidak layak)

In [ ]:
import glob
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import os
import pandas as pd

# Path folder
base_path = "/content/drive/MyDrive/Data Riset/Data Patch/124"

# Ambil semua file .tif dan urutkan
multiband_files = sorted(glob.glob(f"{base_path}/MULTIBAND/*.tif"))
mask_files = sorted(glob.glob(f"{base_path}/MASK/*.tif"))

# Validasi jumlah file
jumlah_patch = min(len(multiband_files), len(mask_files))
print(f"Total set citra tersedia: {jumlah_patch} set.\n")

# Fungsi untuk mengecek apakah >30% piksel B5-B4-B3 bernilai 0
def is_black_dominated(multiband_path, threshold=0.3):
    with rasterio.open(multiband_path) as src:
        try:
            img = src.read([5, 4, 3])
            total_pixels = img.shape[1] * img.shape[2]
            black_pixels = np.sum(np.all(img == 0, axis=0))
            return (black_pixels / total_pixels) > threshold
        except:
            return False

# Fungsi untuk menampilkan MULTIBAND dan MASK
def tampilkan_multiband_mask(multiband_path, mask_path, idx):
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    name_multi = os.path.basename(multiband_path)
    name_mask = os.path.basename(mask_path)

    # MULTIBAND (B5-4-3)
    with rasterio.open(multiband_path) as src:
        try:
            img = src.read([5, 4, 3])
            img = np.transpose(img, (1, 2, 0))
            axs[0].imshow(img / np.clip(img.max(), 1e-5, None))
            axs[0].set_title(f"MULTIBAND \n{name_multi}", fontsize=9)
        except:
            axs[0].imshow(np.zeros((100, 100)))
            axs[0].set_title(f"ERROR B5-4-3\n{name_multi}", fontsize=9)

    # MASK
    with rasterio.open(mask_path) as src:
        mask = src.read(1)
        axs[1].imshow(mask, cmap='gray')
        axs[1].set_title(f"MASK\n{name_mask}", fontsize=9)

    for ax in axs:
        ax.axis('off')

    plt.suptitle(f"PATHROW 124 - Patch #{idx}", fontsize=12)
    plt.tight_layout()
    plt.show()

# Filter dan tampilkan
filtered_filenames = []
ditampilkan = 0

for idx, (m, k) in enumerate(zip(multiband_files, mask_files), start=1):
    if is_black_dominated(m):
        tampilkan_multiband_mask(m, k, ditampilkan + 1)
        filtered_filenames.append(os.path.basename(m))
        ditampilkan += 1

# DataFrame hasil filter
df_filtered = pd.DataFrame(filtered_filenames, columns=["filename"])
print("\nDataFrame dari citra hasil filter:")
print(df_filtered)

# Tampilkan total
print(f"\nJumlah citra dengan piksel dominan hitam: {ditampilkan} dari {jumlah_patch} total citra.")


Output hidden; open in https://colab.research.google.com to view.

## Menampilkan Citra Layak (setelahfilter)

In [ ]:
import glob
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import os
import pandas as pd
import shutil

# Path folder asli PATHROW 124 (Tanpa Overlap)
base_path = "/content/drive/MyDrive/Data Riset/Data Patch/124"

# Path folder hasil filter
filtered_path = os.path.join(base_path, "FILTERED")
falsecolor_filtered = os.path.join(filtered_path, "FALSECOLOR_FILTERED")
multiband_filtered = os.path.join(filtered_path, "MULTIBAND_FILTERED")
mask_filtered = os.path.join(filtered_path, "MASK_FILTERED")

# Buat folder jika belum ada
os.makedirs(falsecolor_filtered, exist_ok=True)
os.makedirs(multiband_filtered, exist_ok=True)
os.makedirs(mask_filtered, exist_ok=True)

# Ambil file .tif dari masing-masing folder
falsecolor_files = sorted(glob.glob(f"{base_path}/FALSECOLOR/*.tif"))
multiband_files = sorted(glob.glob(f"{base_path}/MULTIBAND/*.tif"))
mask_files = sorted(glob.glob(f"{base_path}/MASK/*.tif"))

# Fungsi untuk cek rasio piksel hitam
def is_black_dominated(multiband_path, threshold=0.3):
    with rasterio.open(multiband_path) as src:
        try:
            img = src.read([5, 4, 3])
            total_pixels = img.shape[1] * img.shape[2]
            black_pixels = np.sum(np.all(img == 0, axis=0))
            black_ratio = black_pixels / total_pixels
            return black_ratio > threshold
        except:
            return True  # Jika gagal baca, anggap tidak lolos

# Proses semua file
filtered_out = []
copied_count = 0

for f, m, k in zip(falsecolor_files, multiband_files, mask_files):
    filename = os.path.basename(m)

    if not is_black_dominated(m):  # Lolos filter
        shutil.copy(f, os.path.join(falsecolor_filtered, os.path.basename(f)))
        shutil.copy(m, os.path.join(multiband_filtered, os.path.basename(m)))
        shutil.copy(k, os.path.join(mask_filtered, os.path.basename(k)))
        copied_count += 1
    else:
        filtered_out.append(filename)

# Tampilkan hasil akhir
print(f"\n Total citra LOLOS filter: {copied_count}")
print(f" Total citra filter: {len(filtered_out)}")

# Hitung jumlah file dalam folder hasil filter
def count_filtered(folder_path):
    return len(glob.glob(f"{folder_path}/*.tif"))

print("\n Jumlah data dalam folder hasil filter:")
print(f"FALSECOLOR_FILTERED:  {count_filtered(falsecolor_filtered)} file")
print(f"MULTIBAND_FILTERED:   {count_filtered(multiband_filtered)} file")
print(f"MASK_FILTERED:        {count_filtered(mask_filtered)} file")



 Total citra LOLOS filter: 504
 Total citra filter: 144

 Jumlah data dalam folder hasil filter:
FALSECOLOR_FILTERED:  504 file
MULTIBAND_FILTERED:   504 file
MASK_FILTERED:        504 file


# Overlap 50% Pathrow 125

## Cek Data memiliki citra tidak layak (Piksel bernilai hitam dominan lebih dari 30%)

In [ ]:
import glob
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import os
import pandas as pd

# Path ke folder Multiband
multiband_path = "/content/drive/MyDrive/Data Riset/Data Patch/125/MULTIBAND"
multiband_files = sorted(glob.glob(f"{multiband_path}/*.tif"))

# Inisialisasi hasil
categories = {
    "<30%": 0,
    "30-50%": 0,
    "50-70%": 0,
    ">70%": 0
}

# List data detail
data_list = []

# Proses semua file
for file in multiband_files:
    with rasterio.open(file) as src:
        try:
            img = src.read([5, 4, 3])
            total_pixels = img.shape[1] * img.shape[2]
            black_pixels = np.sum(np.all(img == 0, axis=0))
            black_ratio = black_pixels / total_pixels

            # Tentukan kategori
            if black_ratio < 0.3:
                categories["<30%"] += 1
                cat = "<30%"
            elif black_ratio < 0.5:
                categories["30-50%"] += 1
                cat = "30-50%"
            elif black_ratio < 0.7:
                categories["50-70%"] += 1
                cat = "50-70%"
            else:
                categories[">70%"] += 1
                cat = ">70%"

            # Simpan ke data list
            data_list.append({
                "filename": os.path.basename(file),
                "black_ratio": round(black_ratio * 100, 2),  # persen
                "category": cat
            })
        except:
            print(f" Gagal membaca: {file}")

# Buat DataFrame
df = pd.DataFrame(data_list)

# Tampilkan tabel ringkas
print("\n Statistik Jumlah Citra Berdasarkan Persentase Piksel Dominan Hitam:\n")
stat_table = pd.DataFrame.from_dict(categories, orient='index', columns=['Jumlah Citra'])
stat_table



 Statistik Jumlah Citra Berdasarkan Persentase Piksel Dominan Hitam:



,Jumlah Citra
<30%,504
30-50%,32
50-70%,27
>70%,85


## Menampilkan citra difilter (tidak layak)

In [ ]:
import glob
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import os
import pandas as pd

# Path folder PATHROW 125
base_path = "/content/drive/MyDrive/Data Riset/Data Patch/125"

# Ambil semua file .tif dan urutkan
multiband_files = sorted(glob.glob(f"{base_path}/MULTIBAND/*.tif"))
mask_files = sorted(glob.glob(f"{base_path}/MASK/*.tif"))

# Validasi jumlah file
jumlah_patch = min(len(multiband_files), len(mask_files))
print(f"Total set citra tersedia: {jumlah_patch} set.\n")

# Fungsi untuk mengecek apakah >30% piksel B5-B4-B3 bernilai 0
def is_black_dominated(multiband_path, threshold=0.3):
    with rasterio.open(multiband_path) as src:
        try:
            img = src.read([5, 4, 3])
            total_pixels = img.shape[1] * img.shape[2]
            black_pixels = np.sum(np.all(img == 0, axis=0))
            return (black_pixels / total_pixels) > threshold
        except:
            return False

# Fungsi untuk menampilkan MULTIBAND dan MASK
def tampilkan_multiband_mask(multiband_path, mask_path, idx):
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    name_multi = os.path.basename(multiband_path)
    name_mask = os.path.basename(mask_path)

    # MULTIBAND (B5-4-3)
    with rasterio.open(multiband_path) as src:
        try:
            img = src.read([5, 4, 3])
            img = np.transpose(img, (1, 2, 0))
            axs[0].imshow(img / np.clip(img.max(), 1e-5, None))
            axs[0].set_title(f"MULTIBAND \n{name_multi}", fontsize=9)
        except:
            axs[0].imshow(np.zeros((100, 100)))
            axs[0].set_title(f"ERROR B5-4-3\n{name_multi}", fontsize=9)

    # MASK
    with rasterio.open(mask_path) as src:
        mask = src.read(1)
        axs[1].imshow(mask, cmap='gray')
        axs[1].set_title(f"MASK\n{name_mask}", fontsize=9)

    for ax in axs:
        ax.axis('off')

    plt.suptitle(f"PATHROW 125 - Patch #{idx}", fontsize=12)
    plt.tight_layout()
    plt.show()

# Filter dan tampilkan
filtered_filenames = []
ditampilkan = 0

for idx, (m, k) in enumerate(zip(multiband_files, mask_files), start=1):
    if is_black_dominated(m):
        tampilkan_multiband_mask(m, k, ditampilkan + 1)
        filtered_filenames.append(os.path.basename(m))
        ditampilkan += 1

# DataFrame hasil filter
df_filtered = pd.DataFrame(filtered_filenames, columns=["filename"])
print("\nDataFrame dari citra hasil filter:")
print(df_filtered)

# Tampilkan total
print(f"\nJumlah citra dengan piksel dominan hitam: {ditampilkan} dari {jumlah_patch} total citra.")


Output hidden; open in https://colab.research.google.com to view.

## Menampilkan Citra Layak (setelah difilter)

In [ ]:
import glob
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import os
import pandas as pd
import shutil

# Path folder asli PATHROW 124 (Tanpa Overlap)
base_path = "/content/drive/MyDrive/Data Riset/Data Patch/125"

# Path folder hasil filter
filtered_path = os.path.join(base_path, "FILTERED")
falsecolor_filtered = os.path.join(filtered_path, "FALSECOLOR_FILTERED")
multiband_filtered = os.path.join(filtered_path, "MULTIBAND_FILTERED")
mask_filtered = os.path.join(filtered_path, "MASK_FILTERED")

# Buat folder jika belum ada
os.makedirs(falsecolor_filtered, exist_ok=True)
os.makedirs(multiband_filtered, exist_ok=True)
os.makedirs(mask_filtered, exist_ok=True)

# Ambil file .tif dari masing-masing folder
falsecolor_files = sorted(glob.glob(f"{base_path}/FALSECOLOR/*.tif"))
multiband_files = sorted(glob.glob(f"{base_path}/MULTIBAND/*.tif"))
mask_files = sorted(glob.glob(f"{base_path}/MASK/*.tif"))

# Fungsi untuk cek rasio piksel hitam
def is_black_dominated(multiband_path, threshold=0.3):
    with rasterio.open(multiband_path) as src:
        try:
            img = src.read([5, 4, 3])
            total_pixels = img.shape[1] * img.shape[2]
            black_pixels = np.sum(np.all(img == 0, axis=0))
            black_ratio = black_pixels / total_pixels
            return black_ratio > threshold
        except:
            return True  # Jika gagal baca, anggap tidak lolos

# Proses semua file
filtered_out = []
copied_count = 0

for f, m, k in zip(falsecolor_files, multiband_files, mask_files):
    filename = os.path.basename(m)

    if not is_black_dominated(m):  # Lolos filter
        shutil.copy(f, os.path.join(falsecolor_filtered, os.path.basename(f)))
        shutil.copy(m, os.path.join(multiband_filtered, os.path.basename(m)))
        shutil.copy(k, os.path.join(mask_filtered, os.path.basename(k)))
        copied_count += 1
    else:
        filtered_out.append(filename)

# Tampilkan hasil akhir
print(f"\n Total citra LOLOS filter: {copied_count}")
print(f" Total citra filter: {len(filtered_out)}")

# Hitung jumlah file dalam folder hasil filter
def count_filtered(folder_path):
    return len(glob.glob(f"{folder_path}/*.tif"))

print("\n Jumlah data dalam folder hasil filter:")
print(f"FALSECOLOR_FILTERED:  {count_filtered(falsecolor_filtered)} file")
print(f"MULTIBAND_FILTERED:   {count_filtered(multiband_filtered)} file")
print(f"MASK_FILTERED:        {count_filtered(mask_filtered)} file")



 Total citra LOLOS filter: 504
 Total citra filter: 144

 Jumlah data dalam folder hasil filter:
FALSECOLOR_FILTERED:  504 file
MULTIBAND_FILTERED:   504 file
MASK_FILTERED:        504 file


# Mengabungkan Folder Menjadi 1 Folder

In [16]:
import os
import shutil

def gabungkan_folder(folder_sumber, folder_tujuan):
    """
    Menyalin semua file dari folder sumber ke folder tujuan.
    """
    os.makedirs(folder_tujuan, exist_ok=True)
    for root, _, files in os.walk(folder_sumber):
        for file in files:
            sumber = os.path.join(root, file)
            tujuan = os.path.join(folder_tujuan, file)
            shutil.copy2(sumber, tujuan)  # Menyalin file beserta metadata

# Direktori dasar sumber
base_path = "/content/drive/MyDrive/Data Riset/Data Patch/"
folder_124 = os.path.join(base_path, "124/FILTERED")
folder_125 = os.path.join(base_path, "125/FILTERED")

# Folder sumber per jenis data
mask_124 = os.path.join(folder_124, "MASK_FILTERED")
mask_125 = os.path.join(folder_125, "MASK_FILTERED")

mb_124 = os.path.join(folder_124, "MULTIBAND_FILTERED")
mb_125 = os.path.join(folder_125, "MULTIBAND_FILTERED")

fc_124 = os.path.join(folder_124, "FALSECOLOR_FILTERED")
fc_125 = os.path.join(folder_125, "FALSECOLOR_FILTERED")

# Folder tujuan
tujuan_base = "/content/drive/MyDrive/DATA BURNED AREA LANDSAT 9"
mask_tujuan = os.path.join(tujuan_base, "MASK")
multiband_tujuan = os.path.join(tujuan_base, "MULTIBAND")
falsecolor_tujuan = os.path.join(tujuan_base, "FALSECOLOR")

# Proses penggabungan
gabungkan_folder(mask_124, mask_tujuan)
gabungkan_folder(mask_125, mask_tujuan)

gabungkan_folder(mb_124, multiband_tujuan)
gabungkan_folder(mb_125, multiband_tujuan)

gabungkan_folder(fc_124, falsecolor_tujuan)
gabungkan_folder(fc_125, falsecolor_tujuan)

print("Penggabungan selesai untuk MASK, MULTIBAND, dan FALSECOLOR.")


Penggabungan selesai untuk MASK, MULTIBAND, dan FALSECOLOR.


In [18]:
# Tampilkan jumlah file hasil penggabungan
def hitung_jumlah_file(folder):
    """
    Menghitung jumlah file di dalam suatu folder (termasuk subfolder).
    """
    return sum(len(files) for _, _, files in os.walk(folder))

jumlah_mask = hitung_jumlah_file(mask_tujuan)
jumlah_mb = hitung_jumlah_file(multiband_tujuan)
jumlah_fc = hitung_jumlah_file(falsecolor_tujuan)

print(f"Total citra MASK       : {jumlah_mask}")
print(f"Total citra MULTIBAND  : {jumlah_mb}")
print(f"Total citra FALSECOLOR : {jumlah_fc}")

Total citra MASK       : 1008
Total citra MULTIBAND  : 1008
Total citra FALSECOLOR : 1008


# Menampilkan sebagian citra yang Layak

In [26]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio

# Ganti dengan path ke folder hasil filter di Google Drive
multiband_filtered = "/content/drive/MyDrive/DATA BURNED AREA LANDSAT 9/MULTIBAND"
mask_filtered = "/content/drive/MyDrive/DATA BURNED AREA LANDSAT 9/MASK"

# Ambil file hasil filter
multiband_result = sorted(glob.glob(f"{multiband_filtered}/*.tif"))
mask_result = sorted(glob.glob(f"{mask_filtered}/*.tif"))

# Tampilkan 60 citra
max_display = min(55, len(multiband_result), len(mask_result))

fig, axs = plt.subplots(max_display, 2, figsize=(12, max_display * 3.5))
fig.suptitle('Citra Hasil Filter', fontsize=16)

for i in range(max_display):
    with rasterio.open(multiband_result[i]) as src:
        multiband = src.read([5, 4, 3])
        multiband = np.transpose(multiband, (1, 2, 0))
        multiband = np.clip(multiband, 0, 1)

    with rasterio.open(mask_result[i]) as src:
        mask = src.read(1)

    axs[i, 0].imshow(multiband)
    axs[i, 0].set_title(f"Multiband {os.path.basename(multiband_result[i])}")
    axs[i, 0].axis("off")

    axs[i, 1].imshow(mask, cmap='gray')
    axs[i, 1].set_title(f"Mask {os.path.basename(mask_result[i])}")
    axs[i, 1].axis("off")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


Output hidden; open in https://colab.research.google.com to view.